In [31]:
#pip install langchain

In [33]:
#pip install rapidocr-onnxruntime

In [35]:
#pip install langchain_community

In [37]:
#pip install langchain_huggingface

In [39]:
#pip install langchain_ollama

In [41]:
#pip install pypdf

In [66]:
import pdb

In [68]:
#pip install faiss-cpu

In [90]:
import os
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [92]:
def load_documents():
    root_folder = "/Users/harinivaranasi/Desktop/Research"
    
    loader = DirectoryLoader(
        path=root_folder,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        loader_kwargs={"extract_images": True},
        recursive=True   
    )
    
    documents = loader.load()
    breakpoint()
    return documents

In [94]:
load_documents()

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-02-07T01:49:05+00:00', 'author': '', 'keywords': '', 'moddate': '2023-02-07T01:49:05+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/harinivaranasi/Desktop/Research/sample_document5.pdf', 'total_pages': 26, 'page': 0, 'page_label': '1'}, page_content='Dynamic Arctic weather variability and connectivity\nJun Meng,1, 2 Jingfang Fan,3, 2, ∗ Uma S Bhatt, 4 and J ¨urgen Kurths2, 4, 5\n1School of Science, Beijing University of Posts and Telecommunications, Beijing 100876, China\n2Potsdam Institute for Climate Impact Research, Potsdam 14412, Germany\n3School of Systems Science/Institute of Nonequilibrium Systems,\nBeijing Normal University, Beijing 100875, China\n4Geophysical Institute, University of Alaska Fairbanks, Fairbanks, AK 99775, USA\n5Institute o

In [96]:
def create_retriever(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        separators=["\n\n", "\n", " "]
    )
    texts = text_splitter.split_documents(documents)
    
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        model_kwargs={"device": "cpu"}
    )
    
    db = FAISS.from_documents(texts, embeddings)
    db.save_local("faiss_index")
    
    return db.as_retriever(search_type='mmr', search_kwargs={"k": 5})

In [98]:
def load_faiss_index():
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        model_kwargs={"device": "cpu"}
    )
    
    return FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True).as_retriever(
        search_type='mmr', search_kwargs={"k": 5}
    )

In [80]:
'''def build_retrieval_qa_chain(retriever):
    
   # system_prompt = (
    #    "You are a highly knowledgeable environmental scientist."
     #   " Use the provided context to answer questions factually and concisely."
      #  "The responses should be concise and precise."
       # " If the answer cannot be derived from the context, respond with"
        #" 'The information is not available in the provided context.'"
        #" Do not add any external information or assumptions."
        #" Context: {context}"
   # )
        system_prompt = (
        "You are a highly knowledgeable environmental scientist. "
        "Use the provided context to answer the following four questions factually and concisely. "
        "Respond only using information found in the context. Do not speculate. "
        "If the answer cannot be found in the context, use 'Not available'.\n\n"
        "Questions:\n"
        "1. What is the name of the data source used in the research?\n"
        "2. What are the variables from this source used in the research?\n"
        "3. What is the time period covered by the data?\n"
        "4. What is the geographical location or coverage of the data?\n\n"
        "Provide your answer in the following structured format:\n"
        "{\n"
        "  \"data_source\": <name>,\n"
        "  \"variables\": [<var1>, <var2>, ...],\n"
        "  \"time_period\": <time range>,\n"
        "  \"location\": <region>\n"
        "}\n\n"
        "Context: {context}"
    )
        

    model = OllamaLLM(
        model='llama3.2',
        temperature=0.3,
        max_tokens=512,
        top_p=0.85
    )
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    stuff_chain = create_stuff_documents_chain(model, prompt)

    retrieval_chain = create_retrieval_chain(
        retriever,
        stuff_chain
    )

    return retrieval_chain'''

'def build_retrieval_qa_chain(retriever):\n    \n   # system_prompt = (\n    #    "You are a highly knowledgeable environmental scientist."\n     #   " Use the provided context to answer questions factually and concisely."\n      #  "The responses should be concise and precise."\n       # " If the answer cannot be derived from the context, respond with"\n        #" \'The information is not available in the provided context.\'"\n        #" Do not add any external information or assumptions."\n        #" Context: {context}"\n   # )\n        system_prompt = (\n        "You are a highly knowledgeable environmental scientist. "\n        "Use the provided context to answer the following four questions factually and concisely. "\n        "Respond only using information found in the context. Do not speculate. "\n        "If the answer cannot be found in the context, use \'Not available\'.\n\n"\n        "Questions:\n"\n        "1. What is the name of the data source used in the research?\n"\n  

In [108]:
def build_retrieval_qa_chain_v2(retriever):
    system_prompt = (
        "You are a highly knowledgeable environmental scientist. "
        "Use the provided context to answer the following four questions factually and concisely. "
        "Respond only using information found in the context. Do not speculate. "
        "If the answer cannot be found in the context, use 'Not available'.\n\n"
        "Context: {context}"
    )

    model = OllamaLLM(
        model='llama3.2',
        temperature=0.3,
        max_tokens=512,
        top_p=0.85
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}")
        ]
    )

    stuff_chain = create_stuff_documents_chain(model, prompt)

    retrieval_chain = create_retrieval_chain(
        retriever,
        stuff_chain
    )

    return retrieval_chain


In [102]:
def build_summarization_chain(model):
    """
    Builds a summarization chain using the provided model.

    Args:
        model: An instance of the language model to perform summarization.

    Returns:
        A summarization chain ready to process input texts.
    """
   
    # Define the summarization prompt
    system_prompt = (
        "You are a summarization assistant."
        " Summarize the following text into a concise and coherent summary,"
        " highlighting the most important points. Avoid unnecessary details."
        " Text to summarize: {text}"
    )

    # Create a chat prompt template
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{text}"),
        ]
    )

    # Create the summarization chain
    try:
        summarization_chain = LLMChain(
            llm=model,
            prompt=summarization_prompt
        )
    except Exception as e:
        raise ValueError("Failed to create the summarization chain. Ensure the model is correctly configured.") from e

    return summarization_chain

In [104]:
def create_retriever():
    documents = load_documents()  # Load fresh documents
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2',
        model_kwargs={"device": "cpu"}
    )
    
    retriever = FAISS.from_documents(documents, embeddings).as_retriever(
        search_type='mmr', search_kwargs={"k": 3}
    )
    
    retriever.vectorstore.save_local("faiss_index")  # Save new FAISS index
    return retriever

In [110]:
def classify_query(query):
    """
    Classifies whether the query is for summarization or QA.
    Args:
        query (str): The user's input query.

    Returns:
        str: "summarization" or "qa" based on the query type.
    """
    keywords = ["summarize", "overview", "brief", "condense"]  # Add more if needed
    for keyword in keywords:
        if keyword in query.lower():
            return "summarization"
    return "qa"

def main():
    # Load or create the retriever
    #retriever = load_faiss_index() if os.path.exists("faiss_index") else create_retriever(load_documents())
    if os.path.exists("faiss_index"):
        os.system("rm -r faiss_index")  # Remove the previous FAISS index

    retriever = create_retriever()
    # Build the QA and summarization chains
    qa_chain = build_retrieval_qa_chain_v2(retriever)
    model = OllamaLLM(model='llama3.2', temperature=0.3, max_tokens=256, top_p=0.9)
    summarization_chain = build_summarization_chain(model)

    while True:
        user_query = input("Type your query here (or type 'exit' to quit): ")

        if user_query.lower() == 'exit':
            print("Exiting the chatbot. Goodbye!")
            break

        if user_query:
            # Classify the query
            query_type = classify_query(user_query)

            try:
                if query_type == "summarization":
                    # Handle summarization
                    documents = retriever.get_relevant_documents(user_query)  # Retrieve relevant document(s)
                    if documents:
                        text_to_summarize = documents[0].page_content  # Get the main content
                        summary = summarization_chain.run({"text": text_to_summarize})
                        print("Summary:", summary)
                    else:
                        print("No relevant documents found to summarize.")

                elif query_type == "qa":
                    # Handle QA
                    response = qa_chain.invoke({"input": user_query})

                    # Print the answer
                    if 'answer' in response:
                        print("Response:", response['answer'])

                    # Print source documents
                    if 'context' in response:
                        print("\nSource Documents:")
                        for doc in response['context']:
                            print(f" - Source: {doc.metadata['source']}, Page: {doc.metadata.get('page', 'N/A')}")
                else:
                    print("Unable to classify the query.")

            except Exception as e:
                print("An error occurred while processing your query:", str(e))
        else:
            print("Please enter a valid query.")

if __name__ == "__main__":
    main()


Type your query here (or type 'exit' to quit):  what are the datasets mentioned


Response: Based on the provided context, I can answer the following questions:

1. What datasets are mentioned in the context?

The context mentions two datasets:
- Source Data (specifically Figs. 2-5)
- GitHub repository code (https://github.com/fanjingfang/DAWV)

No specific names of other datasets are provided.

2. Not available

3. Not available

4. Not available

Source Documents:
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 12
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 15
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 18


Type your query here (or type 'exit' to quit):  what are the datasets used in the research


Response: The context does not explicitly mention the specific datasets used in the research. However, based on the information provided, it can be inferred that the datasets are related to Arctic sea ice cover and weather variability.

One dataset mentioned is the "Sea Ice Index", which appears to be a measure of the area with at least 15% ice cover. Another dataset is the "WACF" (Weather And Climate Forecasting) and "WPS" ( Weather Pattern Simulation), which seem to be related to daily sea ice cover data.

Additionally, there are references to other datasets and time series analysis methods, such as:

* [49] Cohen et al. (2020)
* [50] Pincus (1995)
* [51] Richman & Moorman (2000)
* [52] Costa et al. (2005)
* [53] Peng et al. (1995)
* [54] Livina & Lenton (2007)
* [55] Grassberger & Procaccia (1983)
* [56] Wolf et al. (1985)
* [57] Box et al. (2015)
* [58] Stoica et al. (2005)
* [59] Pomeau & Manneville (1980)
* [60] Thompson & Wallace (2001)
* [61] Comiso (2012)
* [62] Day & Hodges (

Type your query here (or type 'exit' to quit):  what are the variables used in the research


...

Response: Here are the variables used in the context:

1. t (days) - time variable
2. a - parameter in equation zx = 0.2 * x + 0.8u
3. x - variable in equations zx = 0.2 * x + 0.8u and zy = 0.2 * y + 0.8u
4. u - variable in equations zx = 0.2 * x + 0.8u and zy = 0.2 * y + 0.8u
5. ut - variable in equation ut + 1 = 3.8ut(1 ut)
6. WACF - absolute auto-correlation function value
7. WPS - normalized power spectrum density value
8. b - period (days) variable
9. c - autocorrelation function of WACF value
10. rd WACF - correlation coefficient of WACF values
11. re WPS - correlation coefficient of WPS values
12. f - year variable
13. AO - Arctic Oscillation index
14. Cov(X,Y) - covariance between variables X and Y
15. Var(X) - variance of variable X
16. C(τ) - autocorrelation function value at time lag τ
17. τ∗ - characteristic time scale for short-range memory
18. γ - exponent in power-law decline of C(τ)
19. WACF - advanced autocorrelation function value (as defined in equation 8)

Note that

Type your query here (or type 'exit' to quit):  what are the sources of dataset mentioned


Response: Based on the provided context, I can answer the following questions:

1. What is the source of the dataset mentioned in Figure 4?

Not available.

2. What is the source of the dataset used for the analysis (Source Data) and other findings of this study?

The data represented in Figs. 2–5 are available as Source Data, but the specific sources of these datasets are not mentioned in the context.

3. What is the source of the C++ and Python codes used for the analysis?

The C++ and Python codes used for the analysis are available on GitHub: https://github.com/fanjingfang/DAWV.

4. What is the source of the dataset mentioned in the introduction (e.g., air temperature at 850hPa pressure level)?

Not available.

Source Documents:
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 12
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 15
 - Source: /Users/harinivaranasi/Desktop/Research/sample_document5.pdf, Page: 18


Type your query here (or type 'exit' to quit):  exit


Exiting the chatbot. Goodbye!
